# Stock Market Sentiment Analysis
This program predicts if the stock of a company will increase or decrease based on top news headlines

In [2]:
!pip install vaderSentiment


In [3]:
!pip install textblob

In [4]:
!pip install scikit-learn

In [5]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [6]:
data_news = pd.read_csv('data\Combined_News_DJIA.csv', encoding='unicode_escape')
data_news

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain's policy of being tough on drugs is ...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [7]:
data_nums = pd.read_csv('data/upload_DJIA_table.csv')
data_nums

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609


In [8]:
news_details = set(data_news['Date'].unique())
len(news_details)

2107

In [9]:
num_details = set(data_nums['Date'].unique())
len(num_details)

1989

In [10]:
shared_dates = news_details.intersection(num_details)
len(shared_dates)

1989

In [11]:
data_news = data_news[data_news['Date'].isin(shared_dates)]
data_news

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain's policy of being tough on drugs is ...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [12]:
data_nums = data_nums[data_nums['Date'].isin(shared_dates)]
data_nums

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609


In [13]:
merged_data = data_nums.merge(data_news, how='inner', on='Date')
merged_data

,Date,Open,High,Low,Close,Volume,Adj Close,Label,Top1,Top2,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,...,"The United States has placed Myanmar, Uzbekist...",S&amp;P revises European Union credit rating t...,India gets $1 billion loan from World Bank for...,U.S. sailors detained by Iran spoke too much u...,Mass fish kill in Vietnam solved as Taiwan ste...,Philippines president Rodrigo Duterte urges pe...,Spain arrests three Pakistanis accused of prom...,"Venezuela, where anger over food shortages is ...",A Hindu temple worker has been killed by three...,Ozone layer hole seems to be healing - US &amp...
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,...,Googles free wifi at Indian railway stations i...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,...,"Escape Tunnel, Dug by Hand, Is Found at Holoca...",The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",...,"US, Canada and Mexico pledge 50% of power from...",There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,...,b'Bank analyst forecast Georgian crisis 2 days...,"b""Georgia confict could set back Russia's US r...",b'War in the Caucasus is as much the pr

In [14]:
merged_data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988],
           dtype='int64', length=1989)

In [15]:
#combine all the news columns
headlines = []
for row in range(0, len(merged_data.index)):
    headlines.append(' '.join( str(x) for x in merged_data.iloc[row, 8:]))#8th column

In [16]:
headlines[-1]

'b"Georgia \'downs two Russian warplanes\' as countries move to brink of war" b\'BREAKING: Musharraf to be impeached.\' b\'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)\' b\'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire\' b"Afghan children raped with \'impunity,\' U.N. official says - this is sick, a three year old was raped and they do nothing" b\'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.\' b"Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO\'s side" b"The \'enemy combatent\' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it." b\'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]\' b\'Did the U.S. Prep Georgia for War with Russia?\'

In [17]:
merged_data['headlines'] = headlines
merged_data

,Date,Open,High,Low,Close,Volume,Adj Close,Label,Top1,Top2,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,headlines
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,1,A 117-year-old woman in Mexico City finally re...,IMF chief backs Athens as permanent Olympic host,...,S&amp;P revises European Union credit rating t...,India gets $1 billion loan from World Bank for...,U.S. sailors detained by Iran spoke too much u...,Mass fish kill in Vietnam solved as Taiwan ste...,Philippines president Rodrigo Duterte urges pe...,Spain arrests three Pakistanis accused of prom...,"Venezuela, where anger over food shortages is ...",A Hindu temple worker has been killed by three...,Ozone layer hole seems to be healing - US &amp...,A 117-year-old woman in Mexico City finally re...
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,1,Jamaica proposes marijuana dispensers for tour...,Stephen Hawking says pollution and 'stupidity'...,...,Mounting evidence suggests 'hobbits' were wipe...,The men who carried out Tuesday's terror attac...,Calls to suspend Saudi Arabia from UN Human Ri...,More Than 100 Nobel Laureates Call Out Greenpe...,British pedophile sentenced to 85 years in US ...,"US permitted 1,200 offshore fracks in Gulf of ...",We will be swimming in ridicule - French beach...,UEFA says no minutes of silence for Istanbul v...,Law Enforcement Sources: Gun Used in Paris Ter...,Jamaica proposes marijuana dispensers for tour...
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,1,Explosion At Airport In Istanbul,Yemeni former president: Terrorism is the offs...,...,The land under Beijing is sinking by as much a...,Car bomb and Anti-Islamic attack on Mosque in ...,Emaciated lions in Taiz Zoo are trapped in blo...,Rupert Murdoch describes Brexit as 'wonderful'...,More than 40 killed in Yemen suicide attacks,Google Found Disastrous Symantec and Norton Vu...,Extremist violence on the rise in Germany: Dom...,BBC News: Labour MPs pass Corbyn no-confidence...,Tiny New Zealand town with 'too many jobs' lau...,Explosion At Airport In Istanbul Yemeni former...
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,1,"2,500 Scientists To Australia: If You Want To ...","The personal details of 112,000 French police ...",...,There is increasing evidence that Australia is...,"Richard Branson, the founder of Virgin Group, ...","37,000-yr-old skull from Borneo reveals surpri...",Palestinians stone Western Wall worshipers; po...,Jean-Claude Juncker asks Farage: Why are you h...,"""Romanians for Remainians"" offering a new home...",Brexit: Gibraltar in talks with Scotland to st...,8 Suicide Bombers Strike Lebanon,Mexico's security forces routinely use 'sexual...,"2,500 Scientists To Australia: If You Want To ..."
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by France, Germany: Cr...",Pakistani clerics declare transgender marriage...,Barclays and RBS shares suspended from trading...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,...,"b""Georgia confict could set back Russia's US r...",b'War in the Caucasus is as much the product o...,"b'""Non-media"" photos of South Osseti

In [18]:
final_data = merged_data.drop(merged_data.iloc[:, 8:-1], axis=1 )
final_data

,Date,Open,High,Low,Close,Volume,Adj Close,Label,headlines
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,1,A 117-year-old woman in Mexico City finally re...
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,1,Jamaica proposes marijuana dispensers for tour...
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,1,Explosion At Airport In Istanbul Yemeni former...
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,1,"2,500 Scientists To Australia: If You Want To ..."
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,0,Barclays and RBS shares suspended from trading...
...,...,...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,1,b'All the experts admit that we should legalis...
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,0,b' U.S. refuses Israel weapons to attack Iran:...
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,0,b'Remember that adorable 9-year-old who sang a...
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,1,b'Why wont America and Nato help us? If they w...


In [19]:
for i, row in final_data.iterrows():
    final_data["headlines"].values[i] = final_data["headlines"].values[i].replace('\' ',' ')
    final_data["headlines"].values[i] = final_data["headlines"].values[i].replace('b\'','')
    final_data["headlines"].values[i] = final_data["headlines"].values[i].replace('"','')
    final_data["headlines"].values[i] = final_data["headlines"].values[i].lower()

In [20]:
final_data.headlines.values[-3]

"remember that adorable 9-year-old who sang at the opening ceremonies? that was fake, too. brussia 'ends georgia operation' if we had no sexual harassment we would have no children... bal-qa'eda is losing support in iraq because of a brutal crackdown on activities it regards as un-islamic - including women buying cucumbers ceasefire in georgia: putin outmaneuvers the west why microsoft and intel tried to kill the xo $100 laptop stratfor: the russo-georgian war and the balance of power    bi'm trying to get a sense of this whole georgia-russia war: vote up if you think georgia started it, or down if you think russia did bthe us military was surprised by the timing and swiftness of the russian military's move into south ossetia and is still trying to sort out what happened, a us defense official said monday u.s. beats war drum as iran dumps the dollar gorbachev: georgian military attacked the south ossetian capital of tskhinvali with multiple rocket launchers designed to devastate large 

### Subjectivity and Polarity in Sentiment Analysis
Subjectivity quantifies the amount of personal opinion and factual information contained in the text. The higher subjectivity means that the text contains personal opinion rather than factual information(0 to 1)
- 0 -> the content is objective (based on facts)
- 1 -> the content is subjective (based on opinions, differs from person to person)

Polarity is the intensity of a viewpoint. It may be both good and bad. If anything is connected with a strong good sensation or emotion, such as admiration, trust, or love, this will affect all other parts of that object’s existence(+1 to -1) 
- +1 -> positive statement
- -1 -> negative statement

In [21]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [22]:
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [23]:
# create 2 new columns 
final_data['Subjectivity'] = final_data['headlines'].apply(getSubjectivity)
final_data['Polarity'] = final_data['headlines'].apply(getPolarity)

In [24]:
final_data

,Date,Open,High,Low,Close,Volume,Adj Close,Label,headlines,Subjectivity,Polarity
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,1,a 117-year-old woman in mexico city finally re...,0.320261,-0.035458
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,1,jamaica proposes marijuana dispensers for tour...,0.382566,0.011243
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,1,explosion at airport in istanbul yemeni former...,0.389617,0.052622
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,1,"2,500 scientists to australia: if you want to ...",0.352649,0.046560
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,0,barclays and rbs shares suspended from trading...,0.352311,-0.014015
...,...,...,...,...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,1,all the experts admit that we should legalise ...,0.375099,0.040677
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,0,b u.s. refuses israel weapons to attack iran: ...,0.364021,0.011398
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,0,remember that adorable 9-year-old who sang at ...,0.518785,-0.046530
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,1,why wont america and nato help us? if they won...,0.395629,0.128732


In [25]:
#get sentiment score
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [26]:
# get the sentiment scores for each date
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range (0, len(final_data['headlines'])):
    SIA = getSIA(final_data['headlines'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    neu.append(SIA['neu'])
    pos.append(SIA['pos'])

In [27]:
SIA

{'neg': 0.229, 'neu': 0.723, 'pos': 0.049, 'compound': -0.9979}

In [28]:
#store the sentiment scores 
final_data['Compound'] = compound
final_data['Negative'] = neg
final_data['Positive'] = pos
final_data['Neutral'] = neu

In [29]:
final_data

,Date,Open,High,Low,Close,Volume,Adj Close,Label,headlines,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141,1,a 117-year-old woman in mexico city finally re...,0.320261,-0.035458,-0.9983,0.212,0.059,0.729
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234,1,jamaica proposes marijuana dispensers for tour...,0.382566,0.011243,-0.9977,0.202,0.061,0.738
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688,1,explosion at airport in istanbul yemeni former...,0.389617,0.052622,-0.9975,0.225,0.091,0.684
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703,1,"2,500 scientists to australia: if you want to ...",0.352649,0.046560,-0.9571,0.132,0.102,0.767
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234,0,barclays and rbs shares suspended from trading...,0.352311,-0.014015,-0.9644,0.148,0.094,0.758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,1,all the experts admit that we should legalise ...,0.375099,0.040677,-0.9882,0.189,0.094,0.718
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,0,b u.s. refuses israel weapons to attack iran: ...,0.364021,0.011398,-0.9809,0.145,0.065,0.790
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727,0,remember that adorable 9-year-old who sang at ...,0.518785,-0.046530,-0.9715,0.128,0.056,0.816
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609,1,why wont america and nato help us? if they won...,0.395629,0.128732,-0.9858,0.191,0.089,0.721


In [30]:
drop_columns = ['Date', 'Adj Close','headlines']
final_data.drop(drop_columns, axis=1, inplace=True)
# final_data

In [31]:
final_data

,Open,High,Low,Close,Volume,Label,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
0,17924.240234,18002.380859,17916.910156,17949.369141,82160000,1,0.320261,-0.035458,-0.9983,0.212,0.059,0.729
1,17712.759766,17930.609375,17711.800781,17929.990234,133030000,1,0.382566,0.011243,-0.9977,0.202,0.061,0.738
2,17456.019531,17704.509766,17456.019531,17694.679688,106380000,1,0.389617,0.052622,-0.9975,0.225,0.091,0.684
3,17190.509766,17409.720703,17190.509766,17409.720703,112190000,1,0.352649,0.046560,-0.9571,0.132,0.102,0.767
4,17355.210938,17355.210938,17063.080078,17140.240234,138740000,0,0.352311,-0.014015,-0.9644,0.148,0.094,0.758
...,...,...,...,...,...,...,...,...,...,...,...,...
1984,11532.070312,11718.280273,11450.889648,11615.929688,159790000,1,0.375099,0.040677,-0.9882,0.189,0.094,0.718
1985,11632.809570,11633.780273,11453.339844,11532.959961,182550000,0,0.364021,0.011398,-0.9809,0.145,0.065,0.790
1986,11781.700195,11782.349609,11601.519531,11642.469727,173590000,0,0.518785,-0.046530,-0.9715,0.128,0.056,0.816
1987,11729.669922,11867.110352,11675.530273,11782.349609,183190000,1,0.395629,0.128732,-0.9858,0.191,0.089,0.721


In [32]:
# create a feature dataset
X = final_data.drop('Label', axis=1)
y = final_data['Label']

In [33]:
X

,Open,High,Low,Close,Volume,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
0,17924.240234,18002.380859,17916.910156,17949.369141,82160000,0.320261,-0.035458,-0.9983,0.212,0.059,0.729
1,17712.759766,17930.609375,17711.800781,17929.990234,133030000,0.382566,0.011243,-0.9977,0.202,0.061,0.738
2,17456.019531,17704.509766,17456.019531,17694.679688,106380000,0.389617,0.052622,-0.9975,0.225,0.091,0.684
3,17190.509766,17409.720703,17190.509766,17409.720703,112190000,0.352649,0.046560,-0.9571,0.132,0.102,0.767
4,17355.210938,17355.210938,17063.080078,17140.240234,138740000,0.352311,-0.014015,-0.9644,0.148,0.094,0.758
...,...,...,...,...,...,...,...,...,...,...,...
1984,11532.070312,11718.280273,11450.889648,11615.929688,159790000,0.375099,0.040677,-0.9882,0.189,0.094,0.718
1985,11632.809570,11633.780273,11453.339844,11532.959961,182550000,0.364021,0.011398,-0.9809,0.145,0.065,0.790
1986,11781.700195,11782.349609,11601.519531,11642.469727,173590000,0.518785,-0.046530,-0.9715,0.128,0.056,0.816
1987,11729.669922,11867.110352,11675.530273,11782.349609,183190000,0.395629,0.128732,-0.9858,0.191,0.089,0.721


In [34]:
# standardising the independent variables
from sklearn.preprocessing import StandardScaler
object = StandardScaler()
scaled_X = object.fit_transform(X)
print(scaled_X)

[[ 1.4208867   1.42277198  1.44270281 ...  1.29120686 -0.28766221
  -1.03584214]
 [ 1.35358963  1.3998819   1.37758105 ...  1.03838296 -0.19476399
  -0.82817952]
 [ 1.27189004  1.32777192  1.29637109 ...  1.61987792  1.19870932
  -2.07415521]
 ...
 [-0.53378535 -0.56098325 -0.56241934 ... -0.83251389 -0.42700954
   0.97156313]
 [-0.55034236 -0.5339505  -0.5389211  ...  0.76027667  1.1058111
  -1.22043113]
 [-0.64503795 -0.56812398 -0.63019859 ...  1.72100748 -0.75215332
  -1.17428388]]


In [36]:
standardised_X = pd.DataFrame(scaled_X, columns=X.columns)
standardised_X

,Open,High,Low,Close,Volume,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
0,1.420887,1.422772,1.442703,1.427308,-0.858904,-0.633416,-1.025767,-0.230201,1.291207,-0.287662,-1.035842
1,1.353590,1.399882,1.377581,1.421142,-0.317157,0.335611,-0.195787,-0.227393,1.038383,-0.194764,-0.828180
2,1.271890,1.327772,1.296371,1.346279,-0.600970,0.445275,0.539601,-0.226458,1.619878,1.198709,-2.074155
3,1.187400,1.233755,1.212072,1.255621,-0.539096,-0.129696,0.431870,-0.037430,-0.731384,1.709650,-0.159044
4,1.239811,1.216370,1.171614,1.169887,-0.256347,-0.134955,-0.644688,-0.071586,-0.326866,1.338057,-0.366707
...,...,...,...,...,...,...,...,...,...,...,...
1984,-0.613222,-0.581417,-0.610244,-0.587647,-0.032172,0.219477,0.327313,-0.182944,0.709712,1.338057,-1.289652
1985,-0.581165,-0.608366,-0.609466,-0.614044,0.210214,0.047180,-0.193044,-0.148788,-0.402713,-0.008968,0.371649
1986,-0.533785,-0.560983,-0.562419,-0.579204,0.114793,2.454226,-1.222543,-0.104806,-0.832514,-0.427010,0.971563
1987,-0.550342,-0.533950,-0.538921,-0.534702,0.217030,0.538770,1.892248,-0.171715,0.760277,1.105811,-1.220431


In [35]:
y

0       1
1       1
2       1
3       1
4       0
       ..
1984    1
1985    0
1986    0
1987    1
1988    0
Name: Label, Length: 1989, dtype: object

In [38]:
#model with standardised data
X_train_st, X_test_st, y_train, y_test = train_test_split(standardised_X, y , test_size=0.2, random_state=42)
len(X_train_st), len(X_test_st), len(y_train), len(y_test)

model_st = LinearDiscriminantAnalysis().fit(X_train_st, y_train)
y_preds_st = model_st.predict(X_test_st)

print(classification_report(y_test, y_preds_st))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       190
           1       0.89      0.98      0.94       208

    accuracy                           0.93       398
   macro avg       0.94      0.93      0.93       398
weighted avg       0.93      0.93      0.93       398



In [40]:
#deploying the model 
import pickle
file_path = r'models/LDA_model_improved.pkl'
pickle.dump(model_st, open(file_path, 'wb'))

In [39]:
X_train_st

,Open,High,Low,Close,Volume,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
807,0.343047,0.322894,0.340071,0.345022,0.472728,1.883398,0.721256,-0.160485,0.229346,2.499284,-1.451167
1939,-1.426819,-1.358701,-1.477851,-1.466909,2.106384,-0.078146,0.130474,-0.209614,0.633865,0.223278,-0.689738
416,1.249532,1.258271,1.274064,1.279421,-0.924187,0.755596,0.511722,-0.203999,0.305194,0.594871,-0.574370
1273,-0.424805,-0.451750,-0.455772,-0.480757,0.165059,-0.334203,1.549300,-0.156742,-0.604972,0.083931,0.510091
543,0.936373,0.950934,0.947546,0.972096,-0.734197,-0.249346,0.672264,-0.225054,0.684430,-0.845052,-0.205192
...,...,...,...,...,...,...,...,...,...,...,...
1130,-0.339430,-0.354428,-0.328827,-0.332339,-0.188296,-0.901055,0.738619,-0.185283,-0.756667,-0.008968,0.694680
1294,-0.226968,-0.252941,-0.252640,-0.265016,0.403292,-0.351558,0.434663,-0.150659,-0.807231,0.130380,0.671606
860,0.127753,0.152601,0.154827,0.173948,-0.366252,-0.676389,0.242828,-0.226926,0.962536,-0.705704,-0.528222
1459,-0.933239,-0.941961,-0.918252,-0.919536,0.049085,-1.244791,0.739843,-0.149256,-0.756667,-0.194764,0.810048


In [63]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

(1591, 398, 1591, 398)

In [64]:
#create and train the model
model = LinearDiscriminantAnalysis().fit(X_train, y_train)


In [65]:
y_preds = model.predict(X_test)

In [134]:
X_test

,Open,High,Low,Close,Volume,Subjectivity,Polarity,Compound,Negative,Positive,Neutral
1173,11951.530273,11951.759766,11630.030273,11657.959961,218290000,0.375440,0.154961,-0.9753,0.163,0.119,0.718
1650,10336.000000,10444.599609,10335.769531,10405.830078,195910000,0.390945,0.016546,-0.9876,0.141,0.053,0.806
843,13900.129883,14104.860352,13880.190430,14075.370117,107010000,0.318527,0.053971,-0.9957,0.186,0.051,0.763
70,17588.810547,17588.810547,17486.269531,17502.589844,84240000,0.289682,-0.012514,-0.9921,0.165,0.067,0.768
755,14923.730469,15025.900391,14858.929688,14988.370117,61000000,0.332967,0.041234,-0.9781,0.121,0.056,0.823
...,...,...,...,...,...,...,...,...,...,...,...
905,12963.379883,12980.190430,12868.259766,12878.129883,117900000,0.378225,0.063896,-0.9914,0.128,0.064,0.809
261,18116.240234,18117.710938,18010.580078,18015.949219,258300000,0.362528,0.056955,-0.9945,0.144,0.048,0.808
514,16806.119141,16911.410156,16755.289062,16906.619141,73720000,0.351039,-0.034111,-0.9940,0.182,0.096,0.722
1721,9542.910156,9543.059570,9436.129883,9496.280273,201600000,0.368227,0.027409,-0.9935,0.154,0.059,0.787


In [66]:
y_preds

array(['0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1',
       '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1',
       '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '0',
       '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '0', '0', '1',
       '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '0', '1',
       '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1',
       '1', '0', '0', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0',
       '0', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0',
       '1', '0', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1',
       '1', '1', '1', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1',
       '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1',
       '0', '1', '0', '1', '0', '1', '1', '0', '1', '0', '1', '1', '0',
       '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '0', '1',
       '1', '0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0

In [68]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       190
           1       0.89      0.98      0.94       208

    accuracy                           0.93       398
   macro avg       0.94      0.93      0.93       398
weighted avg       0.93      0.93      0.93       398



### Saving the model 

In [73]:
import pickle 

In [77]:
file_path = r'models/LDA_saved_model.pkl'
pickle.dump(model, open(file_path, 'wb'))

In [141]:
def processing(headline,open,high,low,close,volume,sample_news):
    headline = re.sub(pattern='[^a-zA-Z]',repl=' ', string=headline)
    headline=headline.lower()
    subjectivity=TextBlob(headline).sentiment.subjectivity
    polarity=TextBlob(headline).sentiment.polarity
    SIA = getSIA(headline)
    compund=SIA['compound']
    neg=SIA['neg']
    neu=SIA['neu']
    pos=SIA['pos']
    x=[open,high,low,close,volume,subjectivity,polarity,compund,neg,pos,neu]
    pred=model.predict([x])[0]
    print("This is Pred:",pred)
def getData():
        open = float(input('Enter the opening price: '))
        high = float(input('Enter the highest price: '))
        low = float(input('Enter the lowest price: '))
        close = float(input('Enter the closing price: '))
        volume = float(input('Enter the volume of stocks: '))
        sample_news = input('Enter the news line: ')
        processing(sample_news,open,high,low,close,volume,sample_news)

getData()
